In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Maranhão - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - Consumo de Cimento (t)
0,2003-1,23.368400,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.213662,0.260097,0.635570,4.315322e+07,2.636001e+06,6.034057,3.872495e+07,43.342
1,2003-2,23.167977,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.207986,0.258964,0.635997,4.320832e+07,2.636851e+06,6.036083,3.874405e+07,28.663
2,2003-3,23.563273,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.202310,0.257352,0.636424,4.326343e+07,2.637701e+06,6.038110,3.876316e+07,27.900
3,2003-4,24.008345,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.196633,0.255929,0.636851,4.331853e+07,2.638551e+06,6.040136,3.878226e+07,25.739
4,2003-5,24.366896,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.190957,0.254109,0.637278,4.337364e+07,2.639401e+06,6.042163,3.880136e+07,30.203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,31.724066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520178,NaN,NaN,NaN,NaN,NaN,102.698
236,2022-9,31.679231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518515,NaN,NaN,NaN,NaN,NaN,98.133
237,2022-10,31.524870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516447,NaN,NaN,NaN,NaN,NaN,94.964
238,2022-11,31.512212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513912,NaN,NaN,NaN,NaN,NaN,97.684


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.550687,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.189788,1.221276,1.327369,1.120107,-1.533048,0.771361,0.866519
188,-0.554816,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.185037,1.236987,1.321661,1.107960,-1.504452,0.745175,0.846505
189,-0.564631,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.180287,1.252847,1.315953,1.095812,-1.475855,0.718990,0.826491
190,-0.596444,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.175536,1.276241,1.310245,1.083664,-1.447259,0.692804,0.806477


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Maranhão - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,-1.197964,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.822290,-0.950106,-2.166110,-1.681167,-0.279527,-2.120335,-1.967581
1,-1.214041,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.825118,-0.970613,-2.129811,-1.663459,-0.262506,-2.086804,-1.938229
2,-1.182333,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.827947,-0.999751,-2.093513,-1.645750,-0.245485,-2.053272,-1.908877
3,-1.146631,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.830775,-1.025491,-2.057214,-1.628041,-0.228464,-2.019741,-1.879525
4,-1.117870,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.833603,-1.058403,-2.020915,-1.610333,-0.211443,-1.986210,-1.850174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.796451,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.431469,0.944351,1.437372,1.197811,-1.616416,0.972153,1.061498
158,0.727411,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.418921,0.952621,1.436068,1.200726,-1.633804,0.973222,1.061628
159,0.657765,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.406373,0.960982,1.434764,1.203641,-1.651193,0.974290,1.061759
160,0.591941,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.393825,0.969363,1.433461,1.206556,-1.668581,0.975359,1.061889


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      42.076
1      30.183
2      40.701
3      39.783
4      38.903
        ...  
157    66.661
158    78.801
159    67.256
160    84.280
161    86.501
Name: Maranhão - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Maranhão - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Goiais-Maranhão - value,Maranhão - IDH,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
126,1.796771,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.858960,0.673675,0.395980,0.831326,0.271134,1.034804,0.988167
127,1.807797,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.892045,0.687412,0.398649,0.850067,0.230720,1.041385,0.998849
128,1.778040,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.925130,0.706891,0.401317,0.868808,0.190305,1.047966,1.009532
129,1.750228,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.958215,0.722784,0.403986,0.887549,0.149891,1.054547,1.020214
130,1.717721,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.991300,0.738712,0.406655,0.906289,0.109477,1.061127,1.030897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.550687,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.189788,1.221276,1.327369,1.120107,-1.533048,0.771361,0.866519
188,-0.554816,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.185037,1.236987,1.321661,1.107960,-1.504452,0.745175,0.846505
189,-0.564631,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.180287,1.252847,1.315953,1.095812,-1.475855,0.718990,0.826491
190,-0.596444,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.175536,1.276241,1.310245,1.083664,-1.447259,0.692804,0.806477


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[274104229, 1617502466, 2602785658, 1230751452, 875716638, 2786586991, 1809566917, 3170042487, 2388043310, 3493005868, 2791709699, 1542463147, 2396093453, 442750931, 3851280562, 1330579347, 1806579204, 1309676404, 3735028023, 1332102403, 1942169661, 3339977714, 1531846909, 1000879599, 2327004783, 1235286665, 1866525774, 3188626400, 1838897772, 3305850136, 1480811085, 2559145555, 2077576274, 1110918561, 394904081, 3971719245, 441586672, 300469680, 2070311812, 2817612534, 822845130, 2354139980, 3040402818, 3088014728, 2645002424, 846063065, 3666745420, 1004829943, 3039619647, 3770903556]


Step: 0 ___________________________________________
val_loss: 140.76254272460938
winner_seed: 274104229


Step: 1 ___________________________________________
val_loss: 138.89279174804688
winner_seed: 1617502466


Step: 2 ___________________________________________
val_loss: 88.96696472167969
winner_seed: 2602785658


Step: 3 ___________________________________________
val_loss: 116.04017639160156


Ste

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 135ms/step - loss: 9224.2354 - val_loss: 605.5814
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 725.4760 - val_loss: 666.1331
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 681.5090 - val_loss: 886.4994
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 962.8547 - val_loss: 809.1863
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 1125.1300 - val_loss: 1277.5392
Epoch 6/10000
4/4 [==============================] - 0s 12ms/step - loss: 861.6202 - val_loss: 645.7095
Epoch 7/10000
4/4 [==============================] - 0s 13ms/step - loss: 555.3833 - val_loss: 472.9978
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 743.6394 - val_loss: 620.9684
Epoch 9/10000
4/4 [==============================] - 0s 13ms/step - loss: 2416.5059 - val_loss: 2202.4199
Epoch 10/10000
4/4 [==============================] - 0s 1

4/4 [==============================] - 0s 12ms/step - loss: 157.1778 - val_loss: 214.8668
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 153.5145 - val_loss: 179.0651
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 133.9626 - val_loss: 185.0564
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 165.7365 - val_loss: 184.4584
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 144.4675 - val_loss: 233.9745
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 155.9787 - val_loss: 189.3738
Epoch 85/10000
4/4 [==============================] - 0s 12ms/step - loss: 138.4317 - val_loss: 270.8531
Epoch 86/10000
4/4 [==============================] - 0s 13ms/step - loss: 194.8392 - val_loss: 235.7060
Epoch 87/10000
4/4 [==============================] - 0s 10ms/step - loss: 177.5084 - val_loss: 170.3082
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - l

4/4 [==============================] - 0s 12ms/step - loss: 150.7455 - val_loss: 194.7529
Epoch 157/10000
4/4 [==============================] - 0s 12ms/step - loss: 148.6728 - val_loss: 171.0790
Epoch 158/10000
4/4 [==============================] - 0s 12ms/step - loss: 148.1379 - val_loss: 205.3737
Epoch 159/10000
4/4 [==============================] - 0s 12ms/step - loss: 149.5814 - val_loss: 175.8714
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 129.7442 - val_loss: 201.3579
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 151.3692 - val_loss: 252.8823
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 170.1600 - val_loss: 175.8454
Epoch 163/10000
4/4 [==============================] - 0s 12ms/step - loss: 138.3023 - val_loss: 165.8474
Epoch 164/10000
4/4 [==============================] - 0s 11ms/step - loss: 141.0210 - val_loss: 177.3542
Epoch 165/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 11ms/step - loss: 127.5314 - val_loss: 172.2408
Epoch 234/10000
4/4 [==============================] - 0s 11ms/step - loss: 143.8712 - val_loss: 208.6445
Epoch 235/10000
4/4 [==============================] - 0s 12ms/step - loss: 137.2125 - val_loss: 182.6303
Epoch 236/10000
4/4 [==============================] - 0s 11ms/step - loss: 137.2161 - val_loss: 182.5977
Epoch 237/10000
4/4 [==============================] - 0s 12ms/step - loss: 152.0939 - val_loss: 179.8972
Epoch 238/10000
4/4 [==============================] - 0s 12ms/step - loss: 151.4494 - val_loss: 173.4087
Epoch 239/10000
4/4 [==============================] - 0s 12ms/step - loss: 149.9510 - val_loss: 206.7750
Epoch 240/10000
4/4 [==============================] - 0s 12ms/step - loss: 137.1067 - val_loss: 208.2162
Epoch 241/10000
4/4 [==============================] - 0s 11ms/step - loss: 149.5727 - val_loss: 187.0527
Epoch 242/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 9ms/step - loss: 134.5071 - val_loss: 172.9182
Epoch 311/10000
4/4 [==============================] - 0s 9ms/step - loss: 135.3504 - val_loss: 167.4619
Epoch 312/10000
4/4 [==============================] - 0s 9ms/step - loss: 147.4536 - val_loss: 169.6000
Epoch 313/10000
4/4 [==============================] - 0s 8ms/step - loss: 131.3658 - val_loss: 192.6547
Epoch 314/10000
4/4 [==============================] - 0s 13ms/step - loss: 134.9435 - val_loss: 187.7016
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 130.9011 - val_loss: 167.3354
Epoch 316/10000
4/4 [==============================] - 0s 10ms/step - loss: 129.3842 - val_loss: 160.9224
Epoch 317/10000
4/4 [==============================] - 0s 9ms/step - loss: 115.6556 - val_loss: 163.1072
Epoch 318/10000
4/4 [==============================] - 0s 8ms/step - loss: 135.8603 - val_loss: 161.6246
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - 

...

Epoch 930/10000
4/4 [==============================] - 0s 11ms/step - loss: 118.2831 - val_loss: 175.3704
Epoch 931/10000
4/4 [==============================] - 0s 11ms/step - loss: 95.0471 - val_loss: 137.4052
Epoch 932/10000
4/4 [==============================] - 0s 11ms/step - loss: 118.3107 - val_loss: 281.3131
Epoch 933/10000
4/4 [==============================] - 0s 11ms/step - loss: 129.4617 - val_loss: 162.3319
Epoch 934/10000
4/4 [==============================] - 0s 11ms/step - loss: 116.0723 - val_loss: 188.1411
Epoch 935/10000
4/4 [==============================] - 0s 11ms/step - loss: 100.8548 - val_loss: 153.5468
Epoch 936/10000
4/4 [==============================] - 0s 10ms/step - loss: 103.9124 - val_loss: 153.0080
Epoch 937/10000
4/4 [==============================] - 0s 10ms/step - loss: 105.6935 - val_loss: 156.5467
Epoch 938/10000
4/4 [==============================] - 0s 10ms/step - loss: 104.0095 - val_loss: 166.2879
Epoch 939/10000
4/4 [==========================

4/4 [==============================] - 0s 10ms/step - loss: 89.8128 - val_loss: 106.6983
Epoch 1008/10000
4/4 [==============================] - 0s 12ms/step - loss: 87.7090 - val_loss: 147.5965
Epoch 1009/10000
4/4 [==============================] - 0s 12ms/step - loss: 92.5394 - val_loss: 115.7548
Epoch 1010/10000
4/4 [==============================] - 0s 11ms/step - loss: 99.8371 - val_loss: 147.0861
Epoch 1011/10000
4/4 [==============================] - 0s 11ms/step - loss: 110.6321 - val_loss: 191.7091
Epoch 1012/10000
4/4 [==============================] - 0s 11ms/step - loss: 112.4367 - val_loss: 137.8434
Epoch 1013/10000
4/4 [==============================] - 0s 12ms/step - loss: 75.1798 - val_loss: 128.9969
Epoch 1014/10000
4/4 [==============================] - 0s 12ms/step - loss: 87.7824 - val_loss: 147.9480
Epoch 1015/10000
4/4 [==============================] - 0s 11ms/step - loss: 85.3977 - val_loss: 147.1773
Epoch 1016/10000
4/4 [==============================] - 0s 12

4/4 [==============================] - 0s 11ms/step - loss: 79.1886 - val_loss: 171.4789
Epoch 1085/10000
4/4 [==============================] - 0s 11ms/step - loss: 75.2556 - val_loss: 157.2721
Epoch 1086/10000
4/4 [==============================] - 0s 10ms/step - loss: 79.9773 - val_loss: 155.4655
Epoch 1087/10000
4/4 [==============================] - 0s 11ms/step - loss: 70.3742 - val_loss: 128.5920
Epoch 1088/10000
4/4 [==============================] - 0s 11ms/step - loss: 71.2591 - val_loss: 181.0206
Epoch 1089/10000
4/4 [==============================] - 0s 11ms/step - loss: 80.1542 - val_loss: 156.5368
Epoch 1090/10000
4/4 [==============================] - 0s 13ms/step - loss: 103.4340 - val_loss: 129.5344
Epoch 1091/10000
4/4 [==============================] - 0s 12ms/step - loss: 87.7907 - val_loss: 115.5912
Epoch 1092/10000
4/4 [==============================] - 0s 12ms/step - loss: 84.7978 - val_loss: 142.7715
Epoch 1093/10000
4/4 [==============================] - 0s 11m

4/4 [==============================] - 0s 10ms/step - loss: 67.4136 - val_loss: 110.1849
Epoch 1162/10000
4/4 [==============================] - 0s 10ms/step - loss: 59.1347 - val_loss: 101.7702
Epoch 1163/10000
4/4 [==============================] - 0s 11ms/step - loss: 47.7395 - val_loss: 105.0325
Epoch 1164/10000
4/4 [==============================] - 0s 11ms/step - loss: 57.0453 - val_loss: 104.8997
Epoch 1165/10000
4/4 [==============================] - 0s 11ms/step - loss: 68.9354 - val_loss: 112.9882
Epoch 1166/10000
4/4 [==============================] - 0s 11ms/step - loss: 62.6059 - val_loss: 110.2398
Epoch 1167/10000
4/4 [==============================] - 0s 11ms/step - loss: 65.8628 - val_loss: 83.6800
Epoch 1168/10000
4/4 [==============================] - 0s 10ms/step - loss: 59.5689 - val_loss: 95.9538
Epoch 1169/10000
4/4 [==============================] - 0s 11ms/step - loss: 69.3459 - val_loss: 98.8941
Epoch 1170/10000
4/4 [==============================] - 0s 11ms/st

4/4 [==============================] - 0s 11ms/step - loss: 75.7728 - val_loss: 125.3199
Epoch 1239/10000
4/4 [==============================] - 0s 11ms/step - loss: 66.3062 - val_loss: 113.5396
Epoch 1240/10000
4/4 [==============================] - 0s 11ms/step - loss: 66.2017 - val_loss: 155.8605
Epoch 1241/10000
4/4 [==============================] - 0s 11ms/step - loss: 82.9096 - val_loss: 192.6236
Epoch 1242/10000
4/4 [==============================] - 0s 10ms/step - loss: 67.1903 - val_loss: 112.8815
Epoch 1243/10000
4/4 [==============================] - 0s 10ms/step - loss: 56.8227 - val_loss: 128.4232
Epoch 1244/10000
4/4 [==============================] - 0s 12ms/step - loss: 52.3803 - val_loss: 133.9458
Epoch 1245/10000
4/4 [==============================] - 0s 11ms/step - loss: 65.8171 - val_loss: 141.6074
Epoch 1246/10000
4/4 [==============================] - 0s 11ms/step - loss: 68.0271 - val_loss: 108.7721
Epoch 1247/10000
4/4 [==============================] - 0s 11ms

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,77.36776,72.292435,71.348114,69.991028,67.287567,54.763958,54.943966,54.518528,60.270935,61.524338,61.421288,61.206139,60.970112,60.702782,60.691872,60.815193,62.948597,63.659641,64.040596,64.343216,64.247322,64.1241,64.433044,64.527367,64.982506,64.746384,64.959518,64.540527,65.123123,64.95192
Target,88.78,86.195,84.121,88.187,79.514,74.061,64.818,44.254,52.169,53.399,45.364,67.103,61.062,73.445,69.471,83.004,77.332,58.591,70.66,50.796,47.834,45.327,54.032,60.657,67.25,70.773,67.01,81.128,81.737,76.841
Error,11.412239,13.902565,12.772888,18.195969,12.226433,19.297039,9.874035,10.264526,8.101936,8.12534,16.057289,5.896858,0.091888,12.742218,8.779129,22.188805,14.383404,5.068642,6.619408,13.547215,16.413322,18.7971,10.401043,3.870365,2.267494,6.026619,2.050484,16.587471,16.613876,11.889084


In [38]:
display(mae)
display(mape)

11.148822

0.17078479

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined